# Préparation des données

Montage google drive et décompression des données de bdappv.zip

In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
#sys.path.insert(0,'/content/drive/My Drive/statapps-main/src/')
!unzip /content/drive/MyDrive/bdappv/bdappv.zip > /dev/null

Mounted at /content/drive/


Importation du fichier dataloader pour pouvoir utiliser ses modules

In [2]:
!cp /content/drive/MyDrive/statapps-main/src/dataloader.py /content

In [13]:
import dataloader as dtld

In [4]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from PIL import Image
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [31]:
label_attribution = dtld.LabelAttribution(path_image_google='/content/bdappv/google/img', 
                                   path_mask_google='/content/bdappv/google/mask',
                                   path_metadata='/content/bdappv/metadata.csv',
                                   colonne_identifiant='identifiant',
                                   path_export_train_test='/content/drive/MyDrive',
                                   path_image_ign='/content/bdappv/ign/img',
                                   path_mask_ign='/content/bdappv/ign/mask',
                                   use_img_google=True,
                                   use_img_ign=False
                                    )

In [32]:
label_attribution.run()

In [35]:
path_train='/content/drive/MyDrive/train_data.csv'
path_test='/content/drive/MyDrive/test_data.csv'

transformed_train_dataset  = dtld.CustomImageDataset(path_train,'/content/bdappv/google/img', transform=transforms.Compose([
                                               transforms.Resize(224),
                                               transforms.ToTensor()
                                           ]))
transformed_test_dataset = dtld.CustomImageDataset(path_test,"/content/bdappv/google/img",
                                                transform=transforms.Compose([
                                               transforms.Resize(224),
                                               transforms.ToTensor(),
                                           ]))

In [36]:
print("Nombre d'images dans le train: {}".format(transformed_train_dataset.__len__()))
print("Nombre d'images dans le test: {}".format(transformed_test_dataset.__len__()))

Nombre d'images dans le train: 23045
Nombre d'images dans le test: 5762


In [37]:
train_dataloader = DataLoader(transformed_train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(transformed_test_dataset, batch_size=64, shuffle=True)

In [38]:
mean_train, std_train = dtld.mean_std(train_dataloader)
print("Moyenne par channel: {}".format(mean_train.tolist()))
print("Ecart-type par channel: {}".format(std_train.tolist()))

Moyenne par channel: [0.35261356830596924, 0.3558049499988556, 0.3049164414405823]
Ecart-type par channel: [0.20973347127437592, 0.19094730913639069, 0.18523310124874115]


In [39]:
mean_test, std_test = dtld.mean_std(test_dataloader)
print("Moyenne par channel: {}".format(mean_test.tolist()))
print("Ecart-type par channel: {}".format(std_test.tolist()))

Moyenne par channel: [0.3499608635902405, 0.3598434329032898, 0.3147274851799011]
Ecart-type par channel: [0.20578326284885406, 0.18819521367549896, 0.18261423707008362]


In [40]:
normalized_train_dataset  = dtld.CustomImageDataset(path_train,'/content/bdappv/google/img', transform=transforms.Compose([
                                               transforms.Resize(224),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean = mean_train.tolist(),
                                                                    std= std_train.tolist()),
                                               transforms.RandomHorizontalFlip(),
                                           ]))

normalized_test_dataset  = dtld.CustomImageDataset(path_test,"/content/bdappv/google/img", transform=transforms.Compose([
                                               transforms.Resize(224),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean = mean_test.tolist(),
                                                                    std= std_test.tolist()),
                                               transforms.RandomHorizontalFlip(),
                                           ]))

train_dataloader = DataLoader(normalized_train_dataset, batch_size=64, shuffle=False)
test_dataloader = DataLoader(normalized_test_dataset, batch_size=64, shuffle=False)

 # Implémentation ResNet18

In [41]:
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add
from keras.models import Sequential
from keras.models import Model
import tensorflow as tf

In [53]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [172]:
class ResnetBlock(Model):
    """
    A standard resnet block.
    """

    def __init__(self, channels: int, down_sample=False):
        """
        channels: same as number of convolution kernels
        """
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3, 3)
        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras), as suggested in [2]
        INIT_SCHEME = "he_normal"

        self.conv_1 = Conv2D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_1 = BatchNormalization()
        self.conv_2 = Conv2D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_2 = BatchNormalization()
        self.merge = Add()

        if self.__down_sample:
            # perform down sampling using stride of 2, according to [1].
            self.res_conv = Conv2D(
                self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same")
            self.res_bn = BatchNormalization()

    def call(self, inputs):
        res = inputs

        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = tf.nn.relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = tf.nn.relu(x)
        return out

class ResNet18(Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = Conv2D(64, (7, 7), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = BatchNormalization()
        self.pool_2 = MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(64)
        self.res_1_2 = ResnetBlock(64)
        self.res_2_1 = ResnetBlock(128, down_sample=True)
        self.res_2_2 = ResnetBlock(128)
        self.res_3_1 = ResnetBlock(256, down_sample=True)
        self.res_3_2 = ResnetBlock(256)
        self.res_4_1 = ResnetBlock(512, down_sample=True)
        self.res_4_2 = ResnetBlock(512)
        self.avg_pool = GlobalAveragePooling2D()
        self.flat = Flatten()
        self.fc = Dense(num_classes, activation="sigmoid")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
            out = res_block(out)
        out = self.avg_pool(out)
        out = self.flat(out)
        out = self.fc(out)
        return out

In [173]:
model = ResNet18(2)
model.build(input_shape = (None,224 ,224 ,3))
#use categorical_crossentropy since the label is one-hot encoded
from keras.optimizers import SGD
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
model.compile(optimizer = "adam",loss='binary_crossentropy', metrics=["accuracy",recall_m, precision_m, f1_m]) 
model.summary()

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "res_net18_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_400 (Conv2D)         multiple                  9472      
                                                                 
 batch_normalization_400 (Ba  multiple                 256       
 tchNormalization)                                               
                                                                 
 max_pooling2d_20 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 resnet_block_160 (ResnetBlo  multiple                 74368     
 ck)                                                             
                                                 

In [174]:
X_train, Y_train=next(iter(train_dataloader))
X_test, Y_test=next(iter(test_dataloader))

In [175]:
X_train_permute=X_train.permute(0, 2, 3,1)
X_test_permute=X_test.permute(0, 2, 3,1)

In [181]:
print(Y_test.numpy())
len(Y)

[0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 0
 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 0]


64

## On transforme le vecteur des labels pour qu'il correspondent au format attendu par le modèle

Un vecteur de 64 lignes (la dimension du batch) et deux colonnes (car on a deux labels possibles pour nos images).

In [179]:
def label_array_formatting(Y):
  formatted_Y = np.zeros((len(Y), 2))
  for i in range(len(Y)):
    if Y[i] == 1:
      formatted_Y[i][0], formatted_Y[i][1] = 0, 1
    else:
      formatted_Y[i][0], formatted_Y[i][1] = 1, 0
  return formatted_Y

## Lancement du modèle

On convertit le tenseur en array numpy pour que le modèle keras puisse le lire

In [180]:
model.fit(x = X_train_permute.numpy(), y = label_array_formatting(Y_train),
	validation_data=(X_test_permute.numpy(), label_array_formatting(Y_test)),
	batch_size=64,
	epochs=5)

Epoch 1/5
1/1 [==============================] - 33s 33s/step - loss: 0.8327 - accuracy: 0.5156 - recall_m: 0.6562 - precision_m: 0.5000 - f1_m: 0.5676 - val_loss: 2082.7620 - val_accuracy: 0.4844 - val_recall_m: 0.4844 - val_precision_m: 0.4844 - val_f1_m: 0.4844
Epoch 2/5
1/1 [==============================] - 25s 25s/step - loss: 1.7409 - accuracy: 0.7188 - recall_m: 0.7188 - precision_m: 0.7188 - f1_m: 0.7187 - val_loss: 900.0911 - val_accuracy: 0.4844 - val_recall_m: 0.4844 - val_precision_m: 0.4844 - val_f1_m: 0.4844
Epoch 3/5
1/1 [==============================] - 23s 23s/step - loss: 0.7205 - accuracy: 0.6094 - recall_m: 0.6562 - precision_m: 0.6269 - f1_m: 0.6412 - val_loss: 3549.1116 - val_accuracy: 0.4844 - val_recall_m: 0.4844 - val_precision_m: 0.4844 - val_f1_m: 0.4844
Epoch 4/5
1/1 [==============================] - 27s 27s/step - loss: 0.5268 - accuracy: 0.8281 - recall_m: 0.7500 - precision_m: 0.8000 - f1_m: 0.7742 - val_loss: 6263.9067 - val_accuracy: 0.4844 - val_rec

In [184]:
loss, accuracy, recall, precision, f1score = model.evaluate(X_test_permute.numpy(), label_array_formatting(Y_test), verbose=0)
print("Loss sur l'échantillon test: {}".format(round(loss,3)))
print("Accuracy sur l'échantillon test: {}".format(round(accuracy,3)))
print("Recall sur l'échantillon test: {}".format(round(recall_m,3)))
print("Precision sur l'échantillon test: {}".format(round(precision_m,3)))
print("F1-score sur l'échantillon test: {}".format(round(f1_m,3)))

Loss sur l'échantillon test: 5075.498
Accuracy sur l'échantillon test: 0.484


TypeError: ignored